
# Training the model

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Conv1D, \
                                    UpSampling1D, Embedding, Multiply, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.losses import binary_crossentropy
from tensorflow_addons.layers import GELU
from scipy.fft import rfft, irfft
from numpy.random import randint

In [ ]:
%matplotlib inline
file_name = 'cDCGAN'
dir_name = file_name + '_dir'
ckpt_dir = dir_name + '/ckpt'
plots_dir = dir_name + '/plots'
os.mkdir(dir_name)
os.mkdir(ckpt_dir)
os.mkdir(plots_dir)

In [ ]:
data = np.load('../../data/processed/full_data.npz')
electrical_data = data['electrical_data']
labels = data['labels']

In [ ]:
electrical_data = np.concatenate((electrical_data, electrical_data[:5]), axis=0)
labels = np.concatenate((labels, labels[:5]), axis=0)

## Define cDCGANMonitor Callback

In [ ]:
class cDCGANMonitor(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 20 == 0:
            print('\nSaving the generator at epoch', epoch)
            self.model.generator.save_weights(ckpt_dir + '/generator_weights_' + str(epoch) + '.ckpt')
            
            print('Saving the discriminator at epoch', epoch)
            self.model.discriminator.save_weights(ckpt_dir + '/discriminator_weights_' + str(epoch) + '.ckpt')
             
            print('Plotting generated samples')
            self.model.save_plots(epoch)

## Define cDCGAN Class

In [ ]:
class cDCGAN(Model):
    def __init__(self):
        super(cDCGAN, self).__init__()
        
        # Constants and Shapes
        self.g_steps = 3
        self.nr_labels = 5
        self.max_labels = 28
        self.noise_length = 144
        self.seq_length = 8760
        self.nr_features = 6
        self.emb_length = 50
        self.gen_input_shape = (self.noise_length, self.nr_features)
        self.discriminator_input_shape = (self.seq_length, self.nr_features)
        
        # The Generator
        self.generator = self.build_generator()
        self.generator.summary()
        
        # The Discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.summary()

        # Build and compile the discriminator
        #self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    def compile(self):
        super(cDCGAN, self).compile()
        self.d_optimizer = Adam(2e-4, 0.5, 0.9)
        self.g_optimizer = Adam(2e-4, 0.5, 0.9)

        
    def build_generator(self):
        # Hyperparameters
        padding = "valid"
        strides = 2
        dropout = 0.2
        momentum = 0.8
        strides = 2

        # Input
        noise = Input(shape=(self.noise_length,))
        labels = Input(shape=(self.nr_labels,), dtype='int32')
        label_embedding = Embedding(self.max_labels, self.emb_length, input_length=self.nr_labels)(labels)
        flatten_embedding = Flatten()(label_embedding)
        dense_embedding = Dense(self.noise_length)(flatten_embedding)
        multiplied_data = Multiply()([noise, dense_embedding])
        dense_data = Dense(self.noise_length * self.nr_features)(multiplied_data)
        gen_input = Reshape(self.gen_input_shape)(dense_data)
        print('\nInput shape for the Generator =', gen_input.shape)

        # Structure
        model = Sequential() # 144

        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=12, filters=1024, padding=padding))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 277
        
        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=4, filters=512, padding=padding))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 551
        
        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=5, filters=256, padding=padding))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 1098
        
        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=4, filters=128, padding=padding))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 2193
        
        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=4, filters=64, padding=padding))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 4383
        
        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=5, filters=32, padding=padding))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 8762
        
        model.add(UpSampling1D())
        model.add(Conv1D(kernel_size=4, filters=32, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout)) # 8761

        model.add(UpSampling1D())
        model.add(Conv1D(self.nr_features, kernel_size=4, padding=padding, strides=strides))
        model.add(Activation("tanh")) # 8760

        print('Generator Structure:')
        model.build((None, *self.gen_input_shape))
        #model.summary()

        # Result
        res = model(gen_input)
        return Model([noise, labels], res, name="generator")
    
    
    def build_discriminator(self):
        # Hyperparameters
        padding = "causal"
        dropout = 0.2
        momentum = 0.8
        strides = 2

        # Input
        elecs = Input(shape=(self.seq_length, self.nr_features))
        labels = Input(shape=(self.nr_labels,), dtype='int32')
        
        label_embedding = Embedding(self.max_labels, self.seq_length // self.nr_labels, input_length=self.nr_labels)(labels)
        flatten_embedding = Flatten()(label_embedding)
        reshaped_embedding = Reshape((self.seq_length, 1))(flatten_embedding)

        discriminator_input = Multiply()([elecs, reshaped_embedding])
        print('\nInput shape for the Discriminator =', discriminator_input.shape)
        
        # Structure
        model = Sequential() # 8760 * 6
        
        model.add(Conv1D(kernel_size=4, filters=32, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout))
        
        model.add(Conv1D(kernel_size=4, filters=64, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout))
        
        model.add(Conv1D(kernel_size=4, filters=128, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout))
        
        model.add(Conv1D(kernel_size=4, filters=256, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout))
        
        model.add(Conv1D(kernel_size=4, filters=512, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout))
        
        model.add(Conv1D(kernel_size=4, filters=1024, padding=padding, strides=strides))
        model.add(BatchNormalization(momentum=momentum))
        model.add(GELU())
        model.add(Dropout(dropout))
        
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        
        print('Discriminator Structure:')
        model.build((None, *self.discriminator_input_shape))
        #model.summary()
        
        # Result
        res = model(discriminator_input)

        return Model([elecs, labels], res, name="discriminator")

    
    def train_step(self, data):
        real_samples, labels = data
        
        # Get the batch size
        batch_size = tf.shape(real_samples)[0]
        
        valid = tf.ones((batch_size, 1), dtype=tf.int32)
        fake = tf.zeros((batch_size, 1), dtype=tf.int32)
        
        # Get the latent vector
        random_latent_vectors = tf.random.normal([batch_size, self.noise_length])

        with tf.GradientTape() as tape:
            # Generate fake data from the latent vector
            gen_samples = self.generator([random_latent_vectors, labels], training=True)
            # Add noise to gen_samples and real_samples for regularization
            to_add = tf.constant([[[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]]]) * 0.02
            additional_noise = tf.random.normal((batch_size, self.seq_length, self.nr_features)) * to_add 
            # Get the logits for the fake data
            fake_vals = self.discriminator([gen_samples + additional_noise, labels], training=True)
            # Get the logits for the real data
            real_vals = self.discriminator([real_samples + additional_noise, labels], training=True)
            # Calculate the discriminator loss using the fake and real image logits
            d_costs = binary_crossentropy(valid, real_vals) + binary_crossentropy(fake, fake_vals)
            d_loss = d_costs * 0.5

        # Get the gradients w.r.t the discriminator loss
        d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
        # Update the weights of the discriminator using the discriminator optimizer
        self.d_optimizer.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

        for i in range(self.g_steps):
            # Train the generator
            # Get the latent vector
            random_latent_vectors = tf.random.normal([batch_size, self.noise_length])
            with tf.GradientTape() as tape:
                # Generate fake data using the generator
                gen_samples = self.generator([random_latent_vectors, labels], training=True)
                # Get the discriminator logits for fake data
                results = self.discriminator([gen_samples, labels], training=True)
                # Compute penalties for steep fft
                difs1_ord1 = gen_samples[:, 1:, 2] - gen_samples[:, :-1, 2]
                fft1_penalty = tf.reduce_mean(tf.abs(difs1_ord1))
                difs10_ord1 = gen_samples[:, 1:, 3] - gen_samples[:, :-1, 3]
                fft10_penalty = tf.abs(tf.reduce_mean(tf.abs((difs10_ord1[:, 1:] - difs10_ord1[:, :-1]))) - 1.3e-6)
                # Calculate the generator loss
                g_loss = -tf.math.log(results) + 100 * fft1_penalty + 10 * fft10_penalty

            # Get the gradients w.r.t the generator loss
            gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
            # Update the weights of the generator using the generator optimizer
            self.g_optimizer.apply_gradients(zip(gen_gradient, self.generator.trainable_variables))
        
        return {"d_loss": d_loss, "g_loss": g_loss}
    
    
    def save_plots(self, nr_batch):
        path = plots_dir + '/' + str(nr_batch)
        os.mkdir(path)
        nr_plots = 10
        noise = np.random.normal(0, 1, (nr_plots, self.noise_length))
        sampled_labels = labels[:nr_plots]
        gen_samples = self.generator.predict([noise, sampled_labels])

        for i in range(nr_plots):
            fig, subplots = plt.subplots(self.nr_features, 1)
            fig.set_size_inches(40, 60)
            for j in range(self.nr_features):
                subplot = subplots[j]
                subplot.plot(gen_samples[i, :, j])
                [t.set_color('red') for t in subplot.yaxis.get_ticklines()]
                [t.set_color('red') for t in subplot.yaxis.get_ticklabels()]
                #subplots[j].axis('off')            
            plt.savefig(path + '/%d.png' % (i))
            plt.close()

## Building the Model

In [ ]:
epochs, batch_size = 200, 32
callback = cDCGANMonitor()
cdcgan = cDCGAN()
cdcgan.compile()

In [ ]:
history = cdcgan.fit(electrical_data, labels, batch_size=batch_size, epochs=epochs, callbacks=[callback])

In [ ]:
h1 = history.history
plt.figure()
plt.plot(history.history['g_loss'])
plt.show()

plt.figure()
plt.plot(history.history['d_loss'])
plt.show()

np.savez_compressed(file_name + '.npz', **history.history)

In [ ]:
path = 'samples_from_model'
nr_samples = 5000
batch_size = 50
final_scores = [-9999] * nr_samples
final_samples = np.zeros((nr_samples, 8760))
sampled_labels = [[randint(0, 3), randint(3, 12), randint(12, 17), randint(17, 20), randint(20, 28)] for _ in range(nr_samples)]
sampled_labels = np.array(sampled_labels)

for k in range(0, 5000, batch_size):
    batch_labels = sampled_labels[k:k + batch_size]
    
    for i in range(nr_trials):
        noise = np.random.normal(0, 1, (batch_size, cdcgan.noise_length))
        gen_samples = cdcgan.generator.predict([noise, batch_labels])
        discriminator_scores = cdcgan.discriminator.predict([gen_samples, batch_labels])
        for j, score in enumerate(discriminator_scores):
            if score > final_scores[k + j]:
                final_scores[k + j] = score
                final_samples[k + j] = gen_samples[j, :, 0]
    print('Finished for k =', k)

In [ ]:
# Remove the spike at the end
for electrical_value in final_samples:
    electrical_value[-30:] = irfft(rfft(electrical_value[-30:])[:5], n=30)

In [ ]:
np.savez_compressed('../../data/processed/cdcgan_generated_data.npz', electrical_data=final_samples, labels=sampled_labels)